In [1]:
if Sys.iswindows()
    username = "Alex"
    pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
    using Pkg
    Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
else
    username = "sergey"
    pathtorepo = "/home/" *username *"/work/repo/dynamical-systems"
    using Pkg
    Pkg.activate(pathtorepo * "/env/integrate/")
end
include("/home/sergey/work/repo/dynamical-systems/system.jl")

  Activating project at `~/work/repo/dynamical-systems/env/integrate`


FHN2_try3_params (generic function with 1 method)

In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems, LinearAlgebra, JLD
E, x, y = 0..30, 0..1, 0..1
box = E × x × y
using CairoMakie, GLMakie

In [3]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58; const ΔU0 = 0.305; 

In [4]:
function distance_two_points_3d(p1, p2)
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    dist = sqrt( (x1-x2)^2 + (y1-y2)^2 + (z1-z2)^2 )
    return dist
end

function calculate_distance(ds, t_integrate)
    
    traj, _ = trajectory(ds, t_integrate)
    fp, ei, _ = fixedpoints(ds, box, jacob_TM_);

    len_traj = length(traj)
    distance = zeros(len_traj)
    if length(fp) == 1
        for i in range(1, len_traj, step = 1)
            distance[i] = distance_two_points_3d(traj[i], fp[1])
        end
        dis = minimum(distance)
    else
        dis = -1;
    end
    return dis
end

calculate_distance (generic function with 1 method)

In [17]:
I0 = -1.70890023462806; U0 = 0.265189299821211
p = [α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [8.34332599395812, 0.738491837768343, 0.438284750927789]
println("last point trajectory:$u0")

last point trajectory:[8.34332599395812, 0.738491837768343, 0.438284750927789]


In [18]:
time = 10000; tt = 1000; tstep = 0.001
integ_set = (alg = Vern9(), adaptive = false, dt = tstep)

(alg = Vern9(; stage_limiter! = trivial_limiter!, step_limiter! = trivial_limiter!, thread = static(false), lazy = true,), adaptive = false, dt = 0.001)

In [19]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    Vern9
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.07993, 3.3, 3.07, 0.75, 0.4, 0.265189299821211, 0.305, 0.3, -1.70890023462806]
 time:          0.0
 state:         [8.34332599395812, 0.738491837768343, 0.438284750927789]


In [20]:
traj, trange = trajectory(ds, time, Δt = 0.001);

In [21]:
fp, ei, _ = fixedpoints(ds, box, jacob_TM_);

In [22]:
ds = CoupledODEs(TM, traj[end], p, diffeq = integ_set)
calculate_distance(ds, 10000)

1.7910412935974927e-15

In [ ]:
#=ts, tf = 1, 550000
indexx,indexy,indexz = 2, 3, 1
lb_size = 35; tck_size = 30;

CairoMakie.activate!()
f = Figure(size = (900, 600))
axis3 = Axis3(f[1, 1], xlabel = "x", ylabel = "y", zlabel = "E",
                xlabelsize = lb_size, ylabelsize = lb_size, zlabelsize = lb_size,
                xticklabelsize = tck_size, yticklabelsize = tck_size, zticklabelsize = tck_size,
                xgridvisible = false, ygridvisible = false, zgridvisible = false)



lines!(axis3, traj[ts:tf, indexx], traj[ts:tf, indexy], traj[ts:tf, indexz], linewidth = 1.0, color = :black)

display(f)=#